In [2]:
import numpy as np
import torch
import base64
import matplotlib.pyplot as plt
from PIL import Image
from io import BytesIO
import warnings
import csv
import ruclip

from torchvision.transforms import Compose, Resize, CenterCrop, ToTensor

csv.field_size_limit(2147483647)

warnings.filterwarnings("ignore", category=UserWarning)
model, processor = ruclip.load("ruclip-vit-base-patch32-384")

def get_image(bs4):
    return Image.open(BytesIO(base64.b64decode(bs4)))

with open('test_v3.csv', 'r', newline='', encoding='utf-8') as csvfile:
    csvreader = csv.reader(csvfile)
    mreader = list(csvreader)

images = [get_image(bs4_url[1]) for bs4_url in mreader[1:]]
y_pred = [bs4_url[2] for bs4_url in mreader[1:]]
# print(y_pred)

classes = ['архитектура', 'зоопарк', 'музей', 'памятник', 'парк', 'пляж',
           'развлекательный центр', 'религия', 'театр', 'фонтан']
templates = ['{}', 'это {}', 'на фото {}']

C:\Users\zarub\PycharmProjects\nto_cv\venv\lib\site-packages\huggingface_hub\file_download.py:655: FutureWarning: 'cached_download' is the legacy way to download files from the HF hub, please consider upgrading to 'hf_hub_download'
  warnings.warn(


In [3]:
predictor = ruclip.Predictor(model, processor, device='cpu', bs=8, templates=templates)
with torch.no_grad():
    text_latents = predictor.get_text_latents(classes)
    pred_labels = predictor.run(images, text_latents)

    # image_latents = predictor.get_image_latents(images)
    # probs = softmax(image_latents, text_latents)

    

490it [02:28,  3.31it/s]


In [6]:
y_test = np.array([classes[pred_label] for pred_label in pred_labels])
print(y_pred[:10])
print(y_test[:10])

['архитектура', 'архитектура', 'архитектура', 'архитектура', 'архитектура', 'архитектура', 'архитектура', 'архитектура', 'архитектура', 'архитектура']
['театр' 'театр' 'театр' 'театр' 'архитектура' 'театр' 'театр'
 'архитектура' 'театр' 'театр']


In [11]:
result = {'архитектура': 0, 'зоопарк': 0, 'музей': 0, 'памятник': 0, 'парк': 0, 'пляж': 0,
           'развлекательный центр': 0, 'религия': 0, 'театр': 0, 'фонтан': 0}
for test, pred in zip(y_test, y_pred):
    if test == pred:
        result[test] += 1

print(result)

{'архитектура': 6, 'зоопарк': 49, 'музей': 40, 'памятник': 47, 'парк': 25, 'пляж': 40, 'развлекательный центр': 41, 'религия': 24, 'театр': 44, 'фонтан': 39}


In [13]:
win_result = {i: y_pred.count(i) for i in y_pred}
print(win_result)

{'архитектура': 49, 'зоопарк': 49, 'музей': 49, 'памятник': 49, 'парк': 49, 'пляж': 49, 'развлекательный центр': 49, 'религия': 49, 'театр': 49, 'фонтан': 49}


In [17]:
for i, key in result.items():
    accuracy = key / win_result[i]
    print(i, round(accuracy, 2))

архитектура 0.12
зоопарк 1.0
музей 0.82
памятник 0.96
парк 0.51
пляж 0.82
развлекательный центр 0.84
религия 0.49
театр 0.9
фонтан 0.8
